## Tarea 2: Limpieza del conjunto de datos

Dentro de este notebook, se trabajo en la depuración de los datos con el fin de facilitar los futuros análisis que se desarrollen a lo largo de este curso.

In [23]:
# Impotación de librerias y lectura del conjunto de datos
import pandas as pd
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
import seaborn as sns

In [46]:
# Leer conjunto de datos
loans = pd.read_csv('C:/Users/AranzaEstebanAvalos/Documents/data.csv', low_memory=False)
# Identificar nombres de columnas
loans.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'pymnt_plan', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'acc_now_delinq', 'tot_col

In [77]:
pd.options.mode.chained_assignment = None  # Desactivar SettingWithCopyWarning

In [78]:
# Lista de columnas_seleccionadas
resultado_columnas = loans.columns.isin([
    'id',
    'loan_amnt',
    'funded_amnt',
    'term',
    'int_rate',
    'installment',
    'grade',
    'sub_grade',
    'emp_length',
    'home_ownership',
    'annual_inc',
    'purpose',
    'addr_state',
    'open_acc',
    'total_pymnt_inv',
    'last_credit_pull_d',
    'earliest_cr_line',
    'total_rec_int',
    'last_pymnt_d',
    'last_pymnt_amnt',
    'last_credit_pull_d',
    'application_type',
    'default_ind'
])
  
columnas_selecc = loans.columns[resultado_columnas]
df = loans[columnas_selecc]

In [79]:
# Validar que el monto del prestamo sea el mismo que se garantizo y en caso de que se igual omitir columna 'funded_amnt'
df['loanVSfunded'] = df['loan_amnt'] - df['funded_amnt']; df['loanVSfunded']

0         0
1         0
2         0
3         0
4         0
         ..
855964    0
855965    0
855966    0
855967    0
855968    0
Name: loanVSfunded, Length: 855969, dtype: int64

In [80]:
# Identificar varibles con NaN
df.isnull().sum()

id                        0
loan_amnt                 0
funded_amnt               0
term                      0
int_rate                  0
installment               0
grade                     0
sub_grade                 0
emp_length            43061
home_ownership            0
annual_inc                0
purpose                   0
addr_state                0
earliest_cr_line          0
open_acc                  0
total_pymnt_inv           0
total_rec_int             0
last_pymnt_d           8862
last_pymnt_amnt           0
last_credit_pull_d       50
application_type          0
default_ind               0
loanVSfunded              0
dtype: int64

In [81]:
# Debido a que la variable con mayor cantidad de NaN´s corresponde al 5% del conjunto de datos, se omitiran dichos datos
df = df.dropna();df

,id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,earliest_cr_line,open_acc,total_pymnt_inv,total_rec_int,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,application_type,default_ind,loanVSfunded
0,1077501,5000,5000,36 months,10.65,162.87,B,B2,10+ years,RENT,...,01-01-1985,3,5831.78,861.07,01-01-2015,171.62,01-01-2016,INDIVIDUAL,0,0
1,1077430,2500,2500,60 months,15.27,59.83,C,C4,< 1 year,RENT,...,01-04-1999,3,1008.71,435.17,01-04-2013,119.66,01-09-2013,INDIVIDUAL,1,0
2,1077175,2400,2400,36 months,15.96,84.33,C,C5,10+ years,RENT,...,01-11-2001,2,3003.65,603.65,01-06-2014,649.91,01-01-2016,INDIVIDUAL,0,0
3,1076863,10000,10000,36 months,13.49,339.31,C,C1,10+ years,RENT,...,01-02-1996,10,12226.30,2209.33,01-01-2015,357.48,01-01-2015,INDIVIDUAL,0,0
4,1075358,3000,3000,60 months,12.69,67.79,B,B5,1 year,RENT,...,01-01-1996,15,3242.17,1009.07,01-01-2016,67.79,01-01-2016,INDIVIDUAL,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855964,36371250,10000,10000,36 months,11.99,332.10,B,B5,8 years,RENT,...,01-09-2004,9,3971.88,1027.35,01-01-2016,332.10,01-01-2016,INDIVIDUAL,0,0
855965,36441262,24000,24000,36 months,11.99,797.03,B,B5,10+ years,MORTGAGE,...,01-03-1974,5,9532.39,2465.65,01-01-2016,797.03,01-01-2016,INDIVIDUAL,0,0
855966,36271333,13000,13000,60 months,15.99,316.07,D,D2,5 years,RENT,...,01-09-2003,9,3769.74,1925.24,01-01-2016,316.07,01-01-2016,INDIVIDUAL,0,0
855967,36490806,12000,12000,60 months,19.99,317.86,E,E3,1 year,RENT,...,01-10-2003,17,3787.67,2234.98,01-01-2016,317.86,01-01-2016,INDIVIDUAL,0,0


Reemplazar valores en colmunas con el fin de obtener valores númericos unicamente

In [82]:
# Omitir palabras en term para obtener valores numericos
df['term'] = df['term'].str.replace(' months', '')

# reemplazar '< 1 year' por 0's y omitir palabras 'years/year'
df['emp_length'] = df['emp_length'].str.replace('< 1 year', '0') 
df['emp_length'] = df['emp_length'].str.replace('+ years', '')
df['emp_length'] = df['emp_length'].str.replace(' years', '')
df['emp_length'] = df['emp_length'].str.replace(' year', '')
df['emp_length']

0         10
1          0
2         10
3         10
4          1
          ..
855964     8
855965    10
855966     5
855967     1
855968    10
Name: emp_length, Length: 804578, dtype: object

In [64]:
df

,id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,earliest_cr_line,open_acc,total_pymnt_inv,total_rec_int,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,application_type,default_ind,loanVSfunded
0,1077501,5000,5000,36,10.65,162.87,B,B2,10,RENT,...,01-01-1985,3,5831.78,861.07,01-01-2015,171.62,01-01-2016,INDIVIDUAL,0,0
1,1077430,2500,2500,60,15.27,59.83,C,C4,0,RENT,...,01-04-1999,3,1008.71,435.17,01-04-2013,119.66,01-09-2013,INDIVIDUAL,1,0
2,1077175,2400,2400,36,15.96,84.33,C,C5,10,RENT,...,01-11-2001,2,3003.65,603.65,01-06-2014,649.91,01-01-2016,INDIVIDUAL,0,0
3,1076863,10000,10000,36,13.49,339.31,C,C1,10,RENT,...,01-02-1996,10,12226.30,2209.33,01-01-2015,357.48,01-01-2015,INDIVIDUAL,0,0
4,1075358,3000,3000,60,12.69,67.79,B,B5,1,RENT,...,01-01-1996,15,3242.17,1009.07,01-01-2016,67.79,01-01-2016,INDIVIDUAL,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855964,36371250,10000,10000,36,11.99,332.10,B,B5,8,RENT,...,01-09-2004,9,3971.88,1027.35,01-01-2016,332.10,01-01-2016,INDIVIDUAL,0,0
855965,36441262,24000,24000,36,11.99,797.03,B,B5,10,MORTGAGE,...,01-03-1974,5,9532.39,2465.65,01-01-2016,797.03,01-01-2016,INDIVIDUAL,0,0
855966,36271333,13000,13000,60,15.99,316.07,D,D2,5,RENT,...,01-09-2003,9,3769.74,1925.24,01-01-2016,316.07,01-01-2016,INDIVIDUAL,0,0
855967,36490806,12000,12000,60,19.99,317.86,E,E3,1,RENT,...,01-10-2003,17,3787.67,2234.98,01-01-2016,317.86,01-01-2016,INDIVIDUAL,0,0


#### Reeclasificación de valores categoricos

In [65]:
# Obtener valores unicos
df['sub_grade'].unique

<bound method Series.unique of 0         B2
1         C4
2         C5
3         C1
4         B5
          ..
855964    B5
855965    B5
855966    D2
855967    E3
855968    B5
Name: sub_grade, Length: 804578, dtype: object>

In [74]:
df_copy  = df

In [83]:
subgrades = df['sub_grade'].unique()

# Definir una función para ordenar las subcalificaciones
def custom_sort_key(subgrade):
    match = re.match(r'([A-Za-z]+)(\d+)', subgrade)
    letter = match.group(1)
    number = int(match.group(2))
    return letter, number

# Ordenar las subcalificaciones usando la función personalizada
sorted_subgrades = sorted(subgrades, key=custom_sort_key)

encoder = LabelEncoder()
encoder.fit(sorted_subgrades)

# Crear una columna 'sub_grade_encoded' con los nuevos valores
df['sub_grade_encoded'] = encoder.transform(df['sub_grade'])

# Crear un diccionario para idetntifacar lo nuevos valores asignados
dict_sub_grade = dict(zip(sorted_subgrades, encoder.transform(sorted_subgrades)))
df['sub_grade'] = df['sub_grade'].map(dict_sub_grade)
print(dict_sub_grade)
print(df)


{'A1': 0, 'A2': 1, 'A3': 2, 'A4': 3, 'A5': 4, 'B1': 5, 'B2': 6, 'B3': 7, 'B4': 8, 'B5': 9, 'C1': 10, 'C2': 11, 'C3': 12, 'C4': 13, 'C5': 14, 'D1': 15, 'D2': 16, 'D3': 17, 'D4': 18, 'D5': 19, 'E1': 20, 'E2': 21, 'E3': 22, 'E4': 23, 'E5': 24, 'F1': 25, 'F2': 26, 'F3': 27, 'F4': 28, 'F5': 29, 'G1': 30, 'G2': 31, 'G3': 32, 'G4': 33, 'G5': 34}
              id  loan_amnt  funded_amnt term  int_rate  installment grade  \
0        1077501       5000         5000   36     10.65       162.87     B   
1        1077430       2500         2500   60     15.27        59.83     C   
2        1077175       2400         2400   36     15.96        84.33     C   
3        1076863      10000        10000   36     13.49       339.31     C   
4        1075358       3000         3000   60     12.69        67.79     B   
...          ...        ...          ...  ...       ...          ...   ...   
855964  36371250      10000        10000   36     11.99       332.10     B   
855965  36441262      24000        

In [84]:
grade = df['grade'].unique()
encoder = LabelEncoder()
categorias_numericas = encoder.fit_transform(grade)

#Obtener la lista de categorías únicas y sus valores numéricos
categorias_unicas = df['grade'].unique()

# Guardar valores correspondientes al número en un diccionario dictionary
dict_grade = dict(zip(categorias_unicas, categorias_numericas))
df['grade'] = df['grade'].map(dict_grade)
print(dict_grade)
print(df)


{'B': 1, 'C': 2, 'A': 0, 'E': 4, 'F': 5, 'D': 3, 'G': 6}
              id  loan_amnt  funded_amnt term  int_rate  installment  grade  \
0        1077501       5000         5000   36     10.65       162.87      1   
1        1077430       2500         2500   60     15.27        59.83      2   
2        1077175       2400         2400   36     15.96        84.33      2   
3        1076863      10000        10000   36     13.49       339.31      2   
4        1075358       3000         3000   60     12.69        67.79      1   
...          ...        ...          ...  ...       ...          ...    ...   
855964  36371250      10000        10000   36     11.99       332.10      1   
855965  36441262      24000        24000   36     11.99       797.03      1   
855966  36271333      13000        13000   60     15.99       316.07      3   
855967  36490806      12000        12000   60     19.99       317.86      4   
855968  36271262      20000        20000   36     11.99       664.20      

In [85]:
state = df['addr_state'].unique()
encoder = LabelEncoder()
categorias_numericas = encoder.fit_transform(state)

# Obtener la lista de categorías únicas y sus valores numéricos
categorias_unicas = df['addr_state'].unique()

# Guardar valores correspondientes al número en un diccionario dictionary
dict_addr_state = dict(zip(categorias_unicas, categorias_numericas))
df['addr_state'] = df['addr_state'].map(dict_addr_state)
print(dict_addr_state)
print(df)

{'AZ': 3, 'GA': 10, 'IL': 14, 'CA': 4, 'OR': 37, 'NC': 27, 'TX': 43, 'VA': 45, 'MO': 24, 'CT': 6, 'UT': 44, 'FL': 9, 'PA': 38, 'MN': 23, 'NY': 34, 'NJ': 31, 'KY': 17, 'OH': 35, 'SC': 40, 'RI': 39, 'LA': 18, 'MA': 19, 'WA': 47, 'WI': 48, 'AL': 1, 'CO': 5, 'KS': 16, 'NV': 33, 'AK': 0, 'MD': 20, 'WV': 49, 'VT': 46, 'MI': 22, 'DC': 7, 'SD': 41, 'NH': 30, 'AR': 2, 'NM': 32, 'MT': 26, 'HI': 11, 'WY': 50, 'OK': 36, 'DE': 8, 'MS': 25, 'TN': 42, 'IA': 12, 'NE': 29, 'ID': 13, 'IN': 15, 'ME': 21, 'ND': 28}
              id  loan_amnt  funded_amnt term  int_rate  installment  grade  \
0        1077501       5000         5000   36     10.65       162.87      1   
1        1077430       2500         2500   60     15.27        59.83      2   
2        1077175       2400         2400   36     15.96        84.33      2   
3        1076863      10000        10000   36     13.49       339.31      2   
4        1075358       3000         3000   60     12.69        67.79      1   
...          ...        .

In [86]:
purpose = df['purpose'].unique()
encoder = LabelEncoder()
categorias_numericas = encoder.fit_transform(purpose)

# Obtener la lista de categorías únicas y sus valores numéricos
categorias_unicas = df['purpose'].unique()

# Guardar valores correspondientes al número en un diccionario dictionary
dict_purpose = dict(zip(categorias_unicas, categorias_numericas))
df['purpose'] = df['purpose'].map(dict_purpose)
print(dict_purpose)
print(df)


{'credit_card': 1, 'car': 0, 'small_business': 11, 'other': 9, 'wedding': 13, 'debt_consolidation': 2, 'home_improvement': 4, 'major_purchase': 6, 'medical': 7, 'moving': 8, 'vacation': 12, 'house': 5, 'renewable_energy': 10, 'educational': 3}
              id  loan_amnt  funded_amnt term  int_rate  installment  grade  \
0        1077501       5000         5000   36     10.65       162.87      1   
1        1077430       2500         2500   60     15.27        59.83      2   
2        1077175       2400         2400   36     15.96        84.33      2   
3        1076863      10000        10000   36     13.49       339.31      2   
4        1075358       3000         3000   60     12.69        67.79      1   
...          ...        ...          ...  ...       ...          ...    ...   
855964  36371250      10000        10000   36     11.99       332.10      1   
855965  36441262      24000        24000   36     11.99       797.03      1   
855966  36271333      13000        13000   60

In [87]:
apptype = df['application_type'].unique()
encoder = LabelEncoder()
categorias_numericas = encoder.fit_transform(apptype)

# Obtener la lista de categorías únicas y sus valores numéricos
categorias_unicas = df['application_type'].unique()

# Guardar valores correspondientes al número en un diccionario dictionary
dict_apptype = dict(zip(categorias_unicas, categorias_numericas))
df['application_type'] = df['application_type'].map(dict_apptype)
print(dict_apptype)
print(df)

{'INDIVIDUAL': 0, 'JOINT': 1}
              id  loan_amnt  funded_amnt term  int_rate  installment  grade  \
0        1077501       5000         5000   36     10.65       162.87      1   
1        1077430       2500         2500   60     15.27        59.83      2   
2        1077175       2400         2400   36     15.96        84.33      2   
3        1076863      10000        10000   36     13.49       339.31      2   
4        1075358       3000         3000   60     12.69        67.79      1   
...          ...        ...          ...  ...       ...          ...    ...   
855964  36371250      10000        10000   36     11.99       332.10      1   
855965  36441262      24000        24000   36     11.99       797.03      1   
855966  36271333      13000        13000   60     15.99       316.07      3   
855967  36490806      12000        12000   60     19.99       317.86      4   
855968  36271262      20000        20000   36     11.99       664.20      1   

        sub_grade emp

In [88]:
homeown = df['home_ownership'].unique()
encoder = LabelEncoder()
categorias_numericas = encoder.fit_transform(homeown)

# Obtener la lista de categorías únicas y sus valores numéricos
categorias_unicas = df['home_ownership'].unique()

# Guardar valores correspondientes al número en un diccionario dictionary
dict_homeown = dict(zip(categorias_unicas, categorias_numericas))
df['home_ownership'] = df['home_ownership'].map(dict_homeown)
print(dict_homeown)
print(df)

{'RENT': 5, 'OWN': 4, 'MORTGAGE': 1, 'OTHER': 3, 'NONE': 2, 'ANY': 0}
              id  loan_amnt  funded_amnt term  int_rate  installment  grade  \
0        1077501       5000         5000   36     10.65       162.87      1   
1        1077430       2500         2500   60     15.27        59.83      2   
2        1077175       2400         2400   36     15.96        84.33      2   
3        1076863      10000        10000   36     13.49       339.31      2   
4        1075358       3000         3000   60     12.69        67.79      1   
...          ...        ...          ...  ...       ...          ...    ...   
855964  36371250      10000        10000   36     11.99       332.10      1   
855965  36441262      24000        24000   36     11.99       797.03      1   
855966  36271333      13000        13000   60     15.99       316.07      3   
855967  36490806      12000        12000   60     19.99       317.86      4   
855968  36271262      20000        20000   36     11.99      

Generar nueva columna con el periodo del prestamo

In [89]:
# Calcular la diferencia de días y giardar en una nueva columna 'history_length'
df['credit_history'] = (pd.to_datetime(df['last_credit_pull_d'], format="%d-%m-%Y") - pd.to_datetime(df['earliest_cr_line'], format="%d-%m-%Y")).dt.days

# Intercambia los valores de 'credit_history' y 'default_ind'
df['credit_history'], df['default_ind'] = df['default_ind'], df['credit_history']
print(df)


              id  loan_amnt  funded_amnt term  int_rate  installment  grade  \
0        1077501       5000         5000   36     10.65       162.87      1   
1        1077430       2500         2500   60     15.27        59.83      2   
2        1077175       2400         2400   36     15.96        84.33      2   
3        1076863      10000        10000   36     13.49       339.31      2   
4        1075358       3000         3000   60     12.69        67.79      1   
...          ...        ...          ...  ...       ...          ...    ...   
855964  36371250      10000        10000   36     11.99       332.10      1   
855965  36441262      24000        24000   36     11.99       797.03      1   
855966  36271333      13000        13000   60     15.99       316.07      3   
855967  36490806      12000        12000   60     19.99       317.86      4   
855968  36271262      20000        20000   36     11.99       664.20      1   

        sub_grade emp_length  home_ownership  ...  

In [91]:
df.to_csv('C:/Users/AranzaEstebanAvalos/Documents/Aprendizaje_Automatico/df2_loan.csv', index=False)

Crear catalogo de diciionarios

In [92]:
print(dict_addr_state)
print(dict_grade)
print(dict_sub_grade)
print(dict_purpose)
print(dict_apptype)
print(dict_homeown)

{'AZ': 3, 'GA': 10, 'IL': 14, 'CA': 4, 'OR': 37, 'NC': 27, 'TX': 43, 'VA': 45, 'MO': 24, 'CT': 6, 'UT': 44, 'FL': 9, 'PA': 38, 'MN': 23, 'NY': 34, 'NJ': 31, 'KY': 17, 'OH': 35, 'SC': 40, 'RI': 39, 'LA': 18, 'MA': 19, 'WA': 47, 'WI': 48, 'AL': 1, 'CO': 5, 'KS': 16, 'NV': 33, 'AK': 0, 'MD': 20, 'WV': 49, 'VT': 46, 'MI': 22, 'DC': 7, 'SD': 41, 'NH': 30, 'AR': 2, 'NM': 32, 'MT': 26, 'HI': 11, 'WY': 50, 'OK': 36, 'DE': 8, 'MS': 25, 'TN': 42, 'IA': 12, 'NE': 29, 'ID': 13, 'IN': 15, 'ME': 21, 'ND': 28}
{'B': 1, 'C': 2, 'A': 0, 'E': 4, 'F': 5, 'D': 3, 'G': 6}
{'A1': 0, 'A2': 1, 'A3': 2, 'A4': 3, 'A5': 4, 'B1': 5, 'B2': 6, 'B3': 7, 'B4': 8, 'B5': 9, 'C1': 10, 'C2': 11, 'C3': 12, 'C4': 13, 'C5': 14, 'D1': 15, 'D2': 16, 'D3': 17, 'D4': 18, 'D5': 19, 'E1': 20, 'E2': 21, 'E3': 22, 'E4': 23, 'E5': 24, 'F1': 25, 'F2': 26, 'F3': 27, 'F4': 28, 'F5': 29, 'G1': 30, 'G2': 31, 'G3': 32, 'G4': 33, 'G5': 34}
{'credit_card': 1, 'car': 0, 'small_business': 11, 'other': 9, 'wedding': 13, 'debt_consolidation': 2

In [112]:
addrstate = pd.DataFrame.from_dict(dict_addr_state, orient='index')

,0
AZ,3
GA,10
IL,14
CA,4
OR,37
NC,27
TX,43
VA,45
MO,24
CT,6


In [118]:
nombre_dict = ['addr_state', 'grade', 'dict_sub_grade', 'purpose', 'apptype', 'homeown']
lista_dic = [dict_addr_state, dict_grade, dict_sub_grade, dict_purpose, dict_apptype, dict_homeown]
# addrstate = addrstate['addrstate']

In [119]:
df_catalogo = pd.DataFrame(columns=['Valor', 'Columna'])
dfs = []

for key, dictionary in enumerate(lista_dic):
    df_temp = pd.DataFrame(list(dictionary.items()), columns=['Columna', 'Valor'])
    
    # Posicion del dic
    df_temp['NombreDict'] = f"Dict_{key}"
    
    # Append 
    dfs.append(df_temp)

df_catalogo = pd.concat(dfs, ignore_index=True)

df_catalogo['NombreDict'] = df_catalogo['NombreDict'].replace({f'Dict_{k}': nombre_dict[k] for k in range(len(nombre_dict))})

df_catalogo

,Columna,Valor,NombreDict
0,AZ,3,addr_state
1,GA,10,addr_state
2,IL,14,addr_state
3,CA,4,addr_state
4,OR,37,addr_state
...,...,...,...
110,OWN,4,homeown
111,MORTGAGE,1,homeown
112,OTHER,3,homeown
113,NONE,2,homeown


In [120]:
df_catalogo.to_csv('C:/Users/AranzaEstebanAvalos/Documents/Aprendizaje_Automatico/df_catalogo.csv', index=False)